In [ ]:
# !pip install shap
# !pip install autogluon
# !pip install category_encoders

In [ ]:
import time
start_time = time.time()

In [ ]:
import numpy as np
import shap
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import cohen_kappa_score
from sklearn.impute import KNNImputer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
import re
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import seaborn as sn 
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, make_scorer, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import category_encoders as ce

pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)
print("Numpy version {}".format(np.__version__))
print("Pandas version {}".format(pd.__version__))

In [ ]:
# path to data file
df = pd.read_csv(path_data+"hotspot_2020_22_cleaned.csv")
print(df.shape)
df.head()

Fixing French notations:

In [ ]:
df.replace("à", 'a', regex=True, inplace=True)
df.replace("à", 'a', regex=True, inplace=True)
df.replace("è", 'e', regex=True, inplace=True)
df.replace("é", 'e', regex=True, inplace=True)
df.replace("ï", 'i', regex=True, inplace=True)
# hotspot_2020_22_df.replace("__", '_', regex=False, inplace=True)
df[df['admin1']=='menaka']

Getting cluster label as target:

In [ ]:
# The cluster labels are in the same order as the dataset is
# therefore, we can directly get the target column from the cluster dataset
df.rename(columns={'priority_level_validated_by_the_clusters': 'target'}, inplace=True)
df.head()

In [ ]:
# Convert Target to numerical
encoder = ce.OrdinalEncoder(mapping=[{'col': 'target', 'mapping': {"LOW": 0, "MEDIUM": 1, "HIGH": 2, "VERY HIGH":3}}] , return_df=True)
df = encoder.fit_transform(df)
print(df["target"].value_counts())

In [ ]:
# filling out the from 'human' column from Hotspot Analysis with 'Human' data from INFORM model
df['human'].fillna(df['Human'], inplace=True)

# removing all the burden + prevalence data + icf
to_drop = ['Target']

In [ ]:
df.drop(to_drop, axis=1, inplace=True)
print(df.shape)
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe(include=[object])

In [ ]:
# checking the columns having object data type 'x'
def is_float(x):
    try:
        float(x)
        return True
    except ValueError:
        return False


print([x for x in df['Food_Insecurity_Probability'].values if is_float(x) == False])
print([x for x in df['Food_Security'].values if is_float(x) == False])
print([x for x in df['Mortality_rate_under-5'].values if is_float(x) == False])

In [ ]:
# replacing 'x' by np.NaN
df.loc[df['Food_Insecurity_Probability'] == 'x', 'Food_Insecurity_Probability'] = np.NaN
df.loc[df['Food_Security'] == 'x', 'Food_Security'] = np.NaN
df.loc[df['Mortality_rate_under-5'] == 'x', 'Mortality_rate_under-5'] = np.NaN

df['Food_Insecurity_Probability'] = df['Food_Insecurity_Probability'].astype(float)
df['Food_Security'] = df['Food_Security'].astype(float)
df['Mortality_rate_under-5'] = df['Mortality_rate_under-5'].astype(float)

In [ ]:
# removing the categorical columns with a lot categories
to_drop = ['ADMIN_2_Admnistratif', 	'admin2_sanitary', 'DS_ADMIN_2_Admnistratif_Burden_Data']
df.drop(to_drop, axis=1, inplace=True)

In [ ]:
# checking integer columns
df.describe(include=['number'])

In [ ]:
# comparing the 50/ 75/ max values in the above table
cols_for_transformation = ['population_totale', 'population_6-59_month', 'idps', 
                           'Population_sans_acces_aux_structures_de_sante']

for col in cols_for_transformation:
    df[col] = np.log(df[col]+1)

print(df.shape)
df.head()

In [ ]:
#dropping duplicates in 2021
df['key'] = df["country"].astype(str) + df['admin1'].astype(str) + df['ADMIN_2_Admnistratif'].astype(str) + df['admin2_sanitary'].astype(str)
df = df.loc[~(df.index.isin([783,781]))]
print(df.shape)
df['key'].value_counts()

###Intersection###

In [ ]:
df = df[df['key'].map(df['key'].value_counts()) >= 3]
print(df.shape)

In [ ]:
df.drop(['key'], axis=1, inplace=True)
df.shape

In [ ]:
# Removing the columns having more than 70% missing values
print(df.shape)
df.drop(df.columns[df.isnull().sum(axis=0)/ df.shape[0] > 0.7], axis=1, inplace=True)

In [ ]:
print(df.shape)
df.columns

Dropping SAM GAM MAM Burdens:

In [ ]:
drop_burdens = ['gam_burden', 'sam_burden', 'mam_burden','GAM_Burden_R_rest', 'SAM_Burden_R_rest', 'MAM_Burden_R_rest']
df.drop(drop_burdens, axis=1, inplace=True)

In [ ]:
print(df.shape)
df.columns

In [ ]:
target_column = 'target'

In [ ]:
# target column has 0 missing values out of total 1102 values
df['target_null'] = df[target_column].isnull()

print(df.groupby('year')['target_null'].count())

print(df.groupby('year')['target_null'].sum())

In [ ]:
df.drop(['target_null'], axis=1, inplace=True)

In [ ]:
# seeing region wise distribution in different years
area_distribution = df.groupby(['year', 'admin1'])['target'].count().reset_index()

In [ ]:
for i,year in enumerate(area_distribution['year'].unique()):
    data = area_distribution.loc[area_distribution['year'] == year, :]
    print("Number of unique admin regions {}".format(len(data['admin1'].unique())))
    print(sorted(data['admin1'].unique()))
    data.reset_index(drop=True, inplace=True)
    data.sort_values(by='target', inplace=True, ascending=False)
    plt.figure(figsize=(40,10))
    plt.bar(data['admin1'], data['target'])
    plt.xticks(rotation=45)
    plt.xlabel('admin1')
    plt.ylabel('Number of rows for {}'.format(year))
    plt.show()

In [ ]:
#Dropping priority level validated by clusters & DS_ADMIN_2_Admnistratif_Burden_Data
df.drop(['DS_ADMIN_2_Admnistratif_Burden_Data'], axis=1, inplace=True)
print(df.shape)

In [ ]:
#Keeping Year as a categorical value in model
categorical_columns = ['year']
numerical_columns = []

for col in df.columns:
  if col != 'year':
    if df[col].dtype == object:
      categorical_columns.append(col)
    elif df[col].dtype == 'float64':
      numerical_columns.append(col)

In [ ]:
categorical_columns

In [ ]:
numerical_columns

In [ ]:
# encoding categorical columns to numerical columns
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()

for var in categorical_columns:
    df[var] = lbl.fit_transform(df[var])

print(df.shape)

In [ ]:
df['year'].value_counts()

In [ ]:
col_list = df.columns.to_list()
missing_list = list(df.isnull().sum())
missing_df = pd.DataFrame({'variable': col_list,
                           'missing_count': missing_list})
missing_df.head(10)

In [ ]:
# exploring correlation amongst columns having numerical values 
corr = df[numerical_columns].corr()

fig, ax = plt.subplots()
fig.set_figheight(60)
fig.set_figwidth(60)
sns.heatmap(df.corr(method='pearson'), annot=True, fmt='.4f', 
            cmap=plt.get_cmap('coolwarm'), cbar=False, ax=ax)
ax.set_yticklabels(ax.get_yticklabels(), rotation="horizontal")
fig.savefig("out.png")

In [ ]:
# dropping features with correlation greater than 0.9 
corr = df[numerical_columns].corr().abs()
# corr.to_csv('C:/Users/mghosh/OneDrive - UvA/Work/WFP_Wasting/Results/corr.csv')
corr_cut_off = 0.9

all_var = corr.where((np.triu(np.ones(corr.shape), k=1) + 
                           np.tril(np.ones(corr.shape), k=-1)).astype(bool))

upper = corr.where((np.triu(np.ones(corr.shape), k=1)).astype(bool))

corr_check_all = [column for column in all_var.columns if any(all_var[column] > corr_cut_off)]
corr_check_upper = [column for column in upper.columns if any(upper[column] > corr_cut_off)]
missing_corr = missing_df[missing_df['variable'].isin(corr_check_all)]
missing_corr

In [ ]:
corr_check_upper

In [ ]:
to_drop = ['population_6-59_month',
 'gam_prevalence',
 'mam_prevalence',
 'facteur_de_correction-incidence_mam',
 'deworming',
 'lack_of_coping_capacity',
 'Socio-Economic_Vulnerability',
 'Institutional',
 'Infrastructure',
 'LACK_OF_COPING_CAPACITY']
to_drop

In [ ]:
df.drop(to_drop, axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
# feature 'year' used for splitting the data into train test
target_column = 'target'

In [ ]:
df[target_column].hist()

In [ ]:
# splitting the set into train and test set using the 'year'
train = df.loc[df['year'] != 2, :].reset_index(drop=True)
test = df.loc[df['year'] == 2, :].reset_index(drop=True)

In [ ]:
#Dropping >70% missing vars of train from both train and test
drop_vars = train.columns[train.isnull().sum(axis=0)/ train.shape[0] == 1]
print(drop_vars)
train.drop(drop_vars, axis=1, inplace=True)
test.drop(drop_vars, axis=1, inplace=True)

In [ ]:
print(train.columns[train.isnull().sum(axis=0)/ train.shape[0]==1])
print(test.columns[test.isnull().sum(axis=0)/ test.shape[0]==1])

In [ ]:
xtrain = train.loc[:, train.columns != 'target']
ytrain = train['target']
xtest = test.loc[:, train.columns != 'target']
ytest = test['target']

In [ ]:
# qwk metric for calculating divergence regarding actual labels
def sklearn_qwk(y_true, y_pred) -> np.float64:
    """
    Function for measuring Quadratic Weighted Kappa with scikit-learn
    
    :param y_true: The ground truth labels
    :param y_pred: The predicted labels
    
    :return The Quadratic Weighted Kappa Score (QWK)
    """
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")

In [ ]:
def one_away_accuracy(y_true, y_pred) -> np.float64:
    """
    Function for measuring 1-away-accuracy with scikit-learn
    
    :param y_true: The ground truth labels
    :param y_pred: The predicted labels
    
    :return 1_away_accuracy (1_accuracy)
    """
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    #df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)

    to_sum_list = []
    
    for i in range(cmx_data.shape[0]):
        to_sum_list.append(cmx_data[i][i])

    for i in range(1, cmx_data.shape[0]):
        to_sum_list.append(cmx_data[i][i - 1])
        to_sum_list.append(cmx_data[i-1][i])
    
    one_away_acc = sum(to_sum_list)/sum(np.concatenate(cmx_data))
    return one_away_acc

In [ ]:
kappa_scorer = make_scorer(sklearn_qwk)
one_acc_scorer = make_scorer(one_away_accuracy)

In [ ]:
# plotting confusion matrix
def print_cmx(y_true, y_pred, label):
    print("Confusion Matrix: {}".format(label))
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)

    plt.figure(figsize=(10, 7))
    sn.heatmap(df_cmx, annot=True)
    plt.show()

In [ ]:
xtrain_copy = xtrain.copy()
ytrain_copy = ytrain.copy()
xtest_copy = xtest.copy()
ytest_copy = ytest.copy()

In [ ]:
model_outputs = []

In [ ]:
# random forest parameter optimization
# since KNN is a distance based algorithm we'll scale the features and then use imputation to fill the values.
# creating a pipeline for all the algorithms
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('rf', RandomForestClassifier(random_state=2021, n_estimators=200))
])

# we have a list of possible values for each hyperparameter
params= { 
    'rf__max_features': ['auto', 'sqrt'],
    'rf__max_depth' : [4,5,6,8],
    'rf__min_samples_split': [2,5,10]
}


In [ ]:
feat = [x for x in xtrain.columns]

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning; 
#Creating RF only on accuracy score as others (one_away and QWK) shouldn't affect much

clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf.fit(xtrain[feat], ytrain)

score = cross_val_score(clf.best_estimator_, xtrain[feat], ytrain, cv=10, scoring='accuracy')

# saving the best model for RandomForest for stacking
model_rf = clf.best_estimator_

model_rf.fit(xtrain[feat], ytrain)

pred = clf.best_estimator_.predict(xtest[feat])

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print("Best parameter settings RF: \n", model_rf)

print_cmx(ytest, pred, "Accuracy as Metric")

#Storing All 3 results for each algo
acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

# 10 fold cross validation is used for hyperparameter tuning using Cohen’s kappa score
# clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

# clf.fit(xtrain[feat], ytrain)

# check how it performs on the cross validation with QWK as metric
score_qwk = cross_val_score(clf.best_estimator_, xtrain[feat], ytrain, cv=10, scoring='accuracy')

model_rf_qwk = clf.best_estimator_

model_rf_qwk.fit(xtrain[feat], ytrain)

pred = clf.best_estimator_.predict(xtest[feat])

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print(model_rf_qwk)

print_cmx(ytest, pred, "QWK as Metric")

model_outputs.append({
    'model': "RandomForest",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})
model_outputs

In [ ]:
# SHAP values are calculated for each class individually - Accuracy
explainer = shap.TreeExplainer(model_rf['rf'])
shap_values = np.array(explainer.shap_values(xtrain[feat]))


# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model
top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf.best_estimator_.named_steps["rf"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

####LGB Starts####

In [ ]:
# lightgbm hyperparameter tuning
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('lgb', lgb.LGBMClassifier(random_state=2021, n_estimators=250))
])

params = {'lgb__max_depth': [4, 6, 8],
          'lgb__num_leaves': [16, 32],
          'lgb__colsample_bytree': [0.5, 0.75, 1],
          'lgb__subsample': [0.75, 0.85, 1]}

In [ ]:
# Making two Lgb models; 1 which does parameter tuning on accuracy other on QWK; Store 3 metrics for both.

# 10 fold cross validation is used for hyperparameter tuning using accuracy

clf_acc = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf_acc.fit(xtrain[feat], ytrain)

score = cross_val_score(clf_acc.best_estimator_, xtrain[feat], ytrain, cv=10, scoring='accuracy')

model_lgb_acc = clf_acc.best_estimator_

model_lgb_acc.fit(xtrain[feat], ytrain)

pred = clf_acc.best_estimator_.predict(xtest[feat])

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10 fold cross validation is used for hyperparameter tuning using QWK
clf_qwk = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf_qwk.fit(xtrain[feat], ytrain)

score_qwk = cross_val_score(clf_qwk.best_estimator_, xtrain[feat], ytrain, cv=10, scoring='accuracy')

model_lgb_qwk = clf_qwk.best_estimator_

model_lgb_qwk.fit(xtrain[feat], ytrain)

pred = clf_qwk.best_estimator_.predict(xtest[feat])

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

In [ ]:
model_outputs

###Accuracy Plots###

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_acc.best_estimator_.named_steps["lgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_lgb_acc['lgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

###QWK Plots###

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_qwk.best_estimator_.named_steps["lgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_lgb_qwk['lgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

In [ ]:
###Accuracy Plots#### using an additional step to use lightgbm's inbuilt capacility to handle missing values
pipe = Pipeline(steps=[
    ('lgb', lgb.LGBMClassifier(random_state=2021, n_estimators=200))
])

params = {'lgb__max_depth': [4, 6, 8],
          'lgb__num_leaves': [16, 32],
          'lgb__colsample_bytree': [0.5, 0.75, 1],
          'lgb__subsample': [0.75, 0.85, 1]}

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning with accuracy
clf_acc = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf_acc.fit(xtrain, ytrain)

score = cross_val_score(clf_acc.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_lgb_noimp = clf_acc.best_estimator_

model_lgb_noimp.fit(xtrain, ytrain)

pred = clf_acc.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_no_imputation_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning with QWK
clf_qwk = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf_qwk.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf_qwk.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_lgb_qwk_noimp = clf_qwk.best_estimator_

model_lgb_qwk_noimp.fit(xtrain, ytrain)

pred = clf_qwk.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_no_imputation_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})



In [ ]:
model_outputs

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_lgb_noimp['lgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_acc.best_estimator_.named_steps["lgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

###QWK###

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_qwk.best_estimator_.named_steps["lgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_lgb_qwk_noimp['lgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

####XGB Starts####

In [ ]:
# xgboost hyperparameter tuning
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('xgb', xgb.XGBClassifier(random_state=2021, n_estimators=200, min_child_weight=3, use_label_encoder=False, eval_metric='mlogloss'))
])

params =  {
        'xgb__max_depth': [3, 5, 8],
        'xgb__subsample': [0.5, 0.7, 0.9],
        'xgb__colsample_bytree': [0.5, 0.7, 0.9],
    }

In [ ]:
ytrain.value_counts()

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning with accuracy
clf_acc = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf_acc.fit(xtrain, ytrain)

score = cross_val_score(clf_acc.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb = clf_acc.best_estimator_

model_xgb.fit(xtrain, ytrain)

pred = clf_acc.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "XGBoost_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning with QWK
clf_qwk = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf_qwk.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf_qwk.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb_qwk = clf_qwk.best_estimator_

model_xgb_qwk.fit(xtrain, ytrain)

pred = clf_qwk.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc_qwk = accuracy_score(ytest, pred)

model_outputs.append({
    'model': "XGBoost_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})


In [ ]:
model_outputs

###Accurcay Plots###

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_acc.best_estimator_.named_steps["xgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_xgb['xgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

###QWK Plots###

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_qwk.best_estimator_.named_steps["xgb"].feature_importances_
data_imp = importances.sort_values('feature_imp', ascending=False)
list_idx = [0,1,2,3,4,5,6,7,8,9,11,12,13,14,15]
data_imp_idx = data_imp.iloc[list_idx]

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=data_imp_idx)
plt.savefig('C:/Users/mghosh/OneDrive - UvA/Work/WFP_Wasting/Results/Hotspot_1205_1away/Charts_Figures/feature_importance_case2.pdf',
           dpi=300, bbox_inches = "tight")

In [ ]:
importances

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_xgb_qwk['xgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

In [ ]:
shap.summary_plot(shap_values[3], features=xtrain[feat], feature_names=feat, show=False)
plt.savefig('C:/Users/mghosh/OneDrive - UvA/Work/WFP_Wasting/Results/Hotspot_1205_1away/Charts_Figures/SHAP_case2_Very High.pdf',
           dpi=300, bbox_inches = "tight")

In [ ]:
# xgboost hyperparameter tuning without imputation
pipe = Pipeline(steps=[
  ('xgb', xgb.XGBClassifier(random_state=2021, n_estimators=200, min_child_weight=3, use_label_encoder=False, eval_metric='mlogloss'))
])

params =  {
        'xgb__max_depth': [3, 5, 8],
        'xgb__subsample': [0.5, 0.7, 0.9],
        'xgb__colsample_bytree': [0.5, 0.7, 0.9],
    }

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning
clf_acc = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf_acc.fit(xtrain, ytrain)

score = cross_val_score(clf_acc.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb_noimp = clf_acc.best_estimator_

model_xgb_noimp.fit(xtrain, ytrain)

pred = clf_acc.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "XGBoost_no_imputation_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning
clf_qwk = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf_qwk.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf_qwk.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb_qwk_noimp = clf_qwk.best_estimator_

model_xgb_qwk_noimp.fit(xtrain, ytrain)

pred = clf_qwk.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "XGBoost_no_imputation_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})



In [ ]:
model_outputs

###Acuracy Plots###

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_acc.best_estimator_.named_steps["xgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_xgb_noimp['xgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

###QWK Plots###

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model 
#Accuracy

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf_qwk.best_estimator_.named_steps["xgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_xgb_qwk_noimp['xgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

####SVM Starts####

In [ ]:
# for svm model we need to scale the numerical column - se define a pipeline
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('svm', SVC(random_state=2021))
])

# defining parameter grid for finding the optimal C value using the training data
params = {'svm__C':[0.01, 10, 100, 1000, 2500, 5000]}

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf.fit(xtrain, ytrain)

score = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_svm = clf.best_estimator_

model_svm.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "SVC_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_svm_qwk = clf.best_estimator_

model_svm_qwk.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "SVC_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

In [ ]:
# svm explainer for SHAP values
svm_explainer = shap.KernelExplainer(model_svm.predict,xtest.iloc[0:20])
svm_shap_values = svm_explainer.shap_values(xtest.iloc[0:20])

In [ ]:
shap.summary_plot(svm_shap_values, xtest.iloc[0:20], plot_type="bar")
shap.summary_plot(svm_shap_values, xtest.iloc[0:20])


In [ ]:
# Summary of performance of all the models
data_df = pd.DataFrame(model_outputs)

In [ ]:
data_df

In [ ]:
pd.DataFrame(model_outputs).to_csv(path_storage + "performance_summary_2022_test_1205_sg_int.csv", index=False)

In [ ]:
xtrain['target'] = ytrain

In [ ]:
# !pip install autogluon

In [ ]:
# AutoML
from autogluon.tabular import TabularPredictor
model = TabularPredictor(label='target', path="predictSAM").fit(xtrain, time_limit=60)

In [ ]:
pred = model.predict(xtest)

In [ ]:
pred = pred.astype(int)

In [ ]:
print_cmx(ytest, pred, "AutoML")

In [ ]:
print("Classification Report for the test set {}".format(classification_report(ytest, pred)))

In [ ]:
xtest['target'] = ytest

In [ ]:
# Summary of performance by trained models using AutoML
output_autogluon_df = model.leaderboard(xtest, silent=True)
output_autogluon_df

In [ ]:
output_autogluon_df.to_csv('performance_summary_2022_test_1205_autogluon_sg.csv', index=False)

In [ ]:
model

In [ ]:
# class AutogluonWrapper:
    def __init__(self, predictor, feature_names):
        self.ag_model = predictor
        self.feature_names = feature_names
    
    def predict_prob(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.feature_names)
        return self.ag_model.predict_proba(X, as_multiclass=True)

In [ ]:
ag_wrapper = AutogluonWrapper(model, feat)
explainer = shap.KernelExplainer(ag_wrapper.predict_prob, xtest[feat].iloc[0:50])

In [ ]:
# explainer for SHAP values
shap_values = explainer.shap_values(xtest[feat].iloc[0:50])

In [ ]:
shap.summary_plot(shap_values[0], xtest[feat].iloc[0:50], plot_type="bar")
shap.summary_plot(shap_values[0], xtest[feat].iloc[0:50])

In [ ]:
xtrain.drop('target', axis=1, inplace=True)
xtest.drop('target', axis=1, inplace=True)

### Stacking###

In [ ]:
# stacking for combining the predictions by RF, LGB, XGB using lightgbm model (lightgbm as the level 2 model or meta model) 
pred_rf = cross_val_predict(model_rf, xtrain, ytrain, cv=10)
pred_xgb = cross_val_predict(model_xgb_qwk_noimp, xtrain, ytrain, cv=10)
pred_lgb = cross_val_predict(model_lgb_qwk, xtrain, ytrain, cv=10)

In [ ]:
predictions = pd.DataFrame()

predictions['rf'] = pred_rf
predictions['xgb'] = pred_xgb
predictions['lgb'] = pred_lgb

In [ ]:
model = lgb.LGBMClassifier(n_estimators=200)

model.fit(predictions, ytrain)

In [ ]:
pred_rf = model_rf.predict(xtest)
pred_xgb = model_xgb_qwk_noimp.predict(xtest)
pred_lgb = model_lgb_qwk.predict(xtest)

In [ ]:
predictions = pd.DataFrame()

predictions['rf'] = pred_rf
predictions['xgb'] = pred_xgb
predictions['lgb'] = pred_lgb

In [ ]:
pred = model.predict(predictions)

In [ ]:
print_cmx(ytest, pred, "stacking")

In [ ]:
print("Classification Report for the test set {}".format(classification_report(ytest, pred)))

In [ ]:
pd.DataFrame(classification_report(ytest, pred, output_dict=True))

In [ ]:
acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

print('Accuracy: ', acc)
print('QWK: ', qwk)
print('One_away_acc: ', one_away_acc)

RUG and RUX

In [ ]:
#Path where rule discovery files are present
import os
os.chdir("C:/Users/...")

In [ ]:
# !pip install --upgrade ortools

In [ ]:
from rulediscovery import RUXClassifier, RUGClassifier
randomState = 42
maxDepth = 3
penpar = 5
solver = 'gurobi' # choose one of 'gurobi' / 'glpk'

In [ ]:
#Replicate RF from Pipleline
"""
Best RF params:
Pipeline(steps=[('scale', MinMaxScaler()),
                ('impute', KNNImputer(n_neighbors=3)),
                ('rf',
                 RandomForestClassifier(max_depth=4, n_estimators=200,
                                        random_state=2021))])
xtrain, ytrain, xtest, ytest, feat
"""

scale_rf = MinMaxScaler()
impute_rf = KNNImputer(n_neighbors=3)

scale_rf.fit(xtrain_copy)
x_train_rf = scale_rf.transform(xtrain_copy)
x_test_rf = scale_rf.transform(xtest_copy)

impute_rf.fit(x_train_rf)
x_train_rf = impute_rf.transform(x_train_rf)
x_test_rf = impute_rf.transform(x_test_rf)


In [ ]:
RF = RandomForestClassifier(max_depth=6, min_samples_split=10, n_estimators=200, random_state=2021) #rf__max_features
RF_fit = RF.fit(x_train_rf, ytrain)
RF_pred = RF_fit.predict(x_test_rf)
acc = accuracy_score(ytest, RF_pred)
qwk = sklearn_qwk(ytest, RF_pred)
one_away_acc = one_away_accuracy(ytest, RF_pred)

#Printing results for check!
print('Accuracy: ' , acc)
print('QWK: ' , qwk)
print('1-away-acc: ' , one_away_acc)

RUX: Rule Extraction

In [ ]:
RUXRF = RUXClassifier(rf=RF,
                      pen_par=penpar,
                      rule_length_cost=True,
                      false_negative_cost=False, 
                      solver=solver)
RUXRF_fit = RUXRF.fit(x_train_rf, ytrain)
RUXRF_pred = RUXRF.predict(x_test_rf)

acc = accuracy_score(ytest, RUXRF_pred)
qwk = sklearn_qwk(ytest, RUXRF_pred)
one_away_acc = one_away_accuracy(ytest, RUXRF_pred)

#Printing results for check!
print('Accuracy: ' , acc)
print('QWK: ' , qwk)
print('1-away-acc: ' , one_away_acc)

In [ ]:
print('Total number of RF rules: ', RUXRF.get_init_num_of_rules())
print('Total number of rules in RUX(RF): ', RUXRF.get_num_of_rules())
print('Total number of missed samples in RUX(RF): ', RUXRF.get_num_of_missed())
print('Training time for RUX(RF)', RUXRF.get_fit_time())
print('Prediction time for RUX(RF)', RUXRF.get_predict_time())

In [ ]:
RUXRF.print_rules()

RUG: Rule Generation

In [ ]:
RUG = RUGClassifier(max_depth=maxDepth,
                        pen_par=penpar,
                        rule_length_cost=True,
                        false_negative_cost=False,
                        solver=solver,
                        random_state=randomState)
RUG_pred = RUG.fit(x_train_rf, ytrain).predict(x_test_rf)

acc = accuracy_score(ytest, RUG_pred)
qwk = sklearn_qwk(ytest, RUG_pred)
one_away_acc = one_away_accuracy(ytest, RUG_pred)

#Printing results for check!
print('Accuracy: ' , acc)
print('QWK: ' , qwk)
print('1-away-acc: ' , one_away_acc)

In [ ]:
print('Total number of rules in RUG: ', RUG.get_num_of_rules())
print('Total number of missed samples in RUG: ', RUG.get_num_of_missed())
print('Training time for RUG', RUG.get_fit_time())
print('Prediction time for RUG', RUG.get_predict_time())

In [ ]:
RUG.print_rules()

In [ ]:
print("--- %s mins ---" % ((time.time() - start_time)/60))